In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
%store -r model
%store -r X
%store -r y
%store -r X_test
%store -r y_test
%store -r CNN_LSTM_model

In [3]:
from tensorflow.keras.models import Model

pretrained_model = model
# Xóa lớp cuối cùng của mô hình đã được huấn luyện trước đó
pretrained_model.pop()

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp phân loại mới
num_new_classes = 5  # Số lớp ứng dụng mới cần phân loại
pretrained_model.add(keras.layers.Dense(num_new_classes, activation='softmax', name='new_dense_layer'))

# Đóng băng trọng số của các lớp trong mô hình đã được huấn luyện trước đó
for layer in pretrained_model.layers[:-1]:
    layer.trainable = False

# In thông tin mô hình mới
pretrained_model.summary()

# Biên dịch mô hình mới
pretrained_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 128, 32)       832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 128, 32)       25632     
                                                                 
 max_pooling2d (MaxPooling2  (None, 10, 64, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 10, 64, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 64, 64)        51264     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 64, 64)        102464    
                                                        

In [4]:
pretrained_model.fit(X, y, epochs=10, batch_size=16)

Epoch 1/10
823/823 [==============================] - 28s 34ms/step - loss: 1.5280 - accuracy: 0.6434
Epoch 2/10
823/823 [==============================] - 29s 35ms/step - loss: 0.5545 - accuracy: 0.7857
Epoch 3/10
823/823 [==============================] - 29s 35ms/step - loss: 0.4958 - accuracy: 0.8043
Epoch 4/10
823/823 [==============================] - 29s 35ms/step - loss: 0.4714 - accuracy: 0.8154
Epoch 5/10
823/823 [==============================] - 30s 36ms/step - loss: 0.4757 - accuracy: 0.8164
Epoch 6/10
823/823 [==============================] - 30s 37ms/step - loss: 0.4747 - accuracy: 0.8195
Epoch 7/10
823/823 [==============================] - 31s 38ms/step - loss: 0.4668 - accuracy: 0.8202
Epoch 8/10
823/823 [==============================] - 30s 37ms/step - loss: 0.4766 - accuracy: 0.8142
Epoch 9/10
823/823 [==============================] - 28s 34ms/step - loss: 0.4711 - accuracy: 0.8179
Epoch 10/10
823/823 [==============================] - 31s 37ms/step - loss: 0.466

In [5]:
import numpy as np

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = pretrained_model.evaluate(X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = pretrained_model.predict(X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 [==============================] - 28s 67ms/step - loss: 0.3620 - accuracy: 0.8775
Độ chính xác trên tập kiểm tra: 0.8774874806404114
412/412 [==============================] - 27s 64ms/step
Ma trận nhầm lẫn:
[[2840    0    0    0    2]
 [   0 1913    7   18   96]
 [   0   13 2446    0  349]
 [   0  322    6 1352   60]
 [  11   87  633    9 3002]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.82      0.94      0.88      2034
           2       0.79      0.87      0.83      2808
           3       0.98      0.78      0.87      1740
           4       0.86      0.80      0.83      3742

    accuracy                           0.88     13166
   macro avg       0.89      0.88      0.88     13166
weighted avg       0.88      0.88      0.88     13166

